In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

# 순환 신경망(Recurrent Neural Network)
  - sequence model
  - 입력과 출력을 시퀀스 단위로 처리하는 모델
  - RNN은 딥 러닝에 있어 가장 기본적인 시퀀스 모델

  - CNN 등의 모델은 은닉층에서 활성화 함수를 지난 값은 오직 출력층 방향으로만 향함
  - 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징  
<img src='https://drive.google.com/uc?id=1ZRgdk7TcRbJeFtlPEEuq1Z_fFCzQvltq' width=30%>
  - x는 입력층의 입력 벡터, y는 출력층의 출력 벡터
  - RNN에서 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드를 셀(cell)이라고 부릅니다
  - 이 셀은 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행하므로 이를 메모리 셀 또는 RNN 셀이라고 표현합니다
  - 은닉층의 메모리 셀은 각각의 시점(time step)에서 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 하고 있습니다.
  - 현재 시점 t에서의 메모리 셀이 갖고있는 값은 과거의 메모리 셀들의 값에 영향을 받은 것임을 의미합니다
  - 메모리 셀이 출력층 방향으로 또는 다음 시점 t+1의 자신에게 보내는 값을 은닉 상태(hidden state)
  - t 시점의 메모리 셀은 t-1 시점의 메모리 셀이 보낸 은닉 상태값을 t 시점의 은닉 상태 계산을 위한 입력값으로 사용합니다.  
  <img src='https://drive.google.com/uc?id=1YvUtTTJgEAE7xxyaOfj5yUMouE6bSKMA' width=50%>  
  - 피드 포워드 신경망에서는 뉴런이라는 단위를 사용했지만, RNN에서는 뉴런이라는 단위보다는 입력층과 출력층에서는 각각 입력 벡터와 출력 벡터, 은닉층에서는 은닉 상태라는 표현을 주로 사용
  - RNN은 입력과 출력의 길이를 다르게 설계 할 수 있으므로 다양한 용도로 사용할 수 있습니다.
  <img src='https://drive.google.com/uc?id=17MhkjIl4Ml3dR8_9YuGFz8j0Y7SGmwWP' width=50%>.
  - 입력과 출력의 길이에 따라서 달라지는 RNN의 다양한 형태


## One-to-many mapping(일대다)
- 하나의 입력에 대해서 여러개의 출력(one-to-many)의 모델은 하나의 이미지 입력에 대해서 사진의 제목을 출력하는 이미지 캡셔닝(Image Captioning) 작업에 사용할 수 있습니다.
- 사진의 제목은 단어들의 나열이므로 시퀀스 출력입니다

## Many-to-one mapping(다대일)
- 단어 시퀀스에 대해서 하나의 출력(many-to-one)을 하는 모델은 입력 문서가 긍정적인지 부정적인지를 판별하는 감성 분류(sentiment classification), 또는 메일이 정상 메일인지 스팸 메일인지 판별하는 스팸 메일 분류(spam detection)에 사용할 수 있습니다.
<img src='https://drive.google.com/uc?id=11c_fFP1xyGJ1FbAIjQm6ACLi10vlBdcp' width=50%>

## Many-to-many(다대다)
- 다 대 다(many-to-many)의 모델의 경우에는 입력 문장으로 부터 대답 문장을 출력하는 챗봇과 입력 문장으로부터 번역된 문장을 출력하는 번역기, 개체명 인식이나 품사 태깅과 같은 작업이 속합니다.  
<img src='https://drive.google.com/uc?id=1MNbb0cnETmAHb01QWjEUo9B3MHvxGnBH' width=50%>

## RNN의 수식화

<img src='https://drive.google.com/uc?id=1BxSZEK0F712SSK0B5WST-aM4LRABgoMI' width=20%>  
- 은닉층: $h_t = tanh(W_x{x_t} + W_h{h_{t-1}}+b)$
- 출력층: $y_t = f(W_{y}h_t + b)$
- $f$는 비선형 활성화 함수
- 단어 벡터의 차원이 $d$
- 은닉 상태의 크기를 $D_h$.   
<img src='https://drive.google.com/uc?id=1YgE2Fhvf3xqrIdTR2VDz1EYVsEJO5Mnz' width=50%>

# 파이썬으로 RNN 구현하기

```
# 아래의 코드는 의사 코드(pseudocode)로 실제 동작하는 코드가 아님.

hidden_state_t = 0 # 초기 은닉 상태를 0(벡터)로 초기화
for input_t in input_length: # 각 시점마다 입력을 받는다.
    output_t = tanh(input_t, hidden_state_t) # 각 시점에 대해서 입력과 은닉 상태를 가지고 연산
    hidden_state_t = output_t # 계산 결과는 현재 시점의 은닉 상태가 된다.
```

In [1]:
import numpy as np

timesteps = 10 # 시점의 수 -> NLP 분야에서는 보통 문장의 길이!(구성하는 단어의 수)
input_size =4 # 입력 차원 -> 단어 벡터 차원
hidden_size = 8 # 메모리 용량

inputs = np.random.random((timesteps, input_size))
hidden_state_t = np.zeros((hidden_size,))

- 시점, 입력의 차원, 은닉 상태의 크기, 그리고 초기 은닉 상태를 정의한 상태
- 현재 초기 은닉 상태는 0의 값을 가지는 벡터로 초기화가 된 상태

In [2]:
print(hidden_state_t) # 8의 크기를 가지는 은닉 상태. 현재는 초기 은닉 상태로 모든 차원이 0의 값을 가짐.

[0. 0. 0. 0. 0. 0. 0. 0.]


- 은닉 상태의 크기를 8로 정의하였으므로 8의 차원을 가지는 0의 값으로 구성된 벡터가 출력

In [3]:
Wx = np.random.random((hidden_size, input_size))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

- 가중치와 편향을 각 크기에 맞게 정의하였습니다.

In [4]:
print(np.shape(Wx)) # 은닉 상태의 크기 * 입력의 차원
print(np.shape(Wh)) # 은닉 상태의 크기 * 은닉 상태의 크기
print(np.shape(b)) # 은닉 상태의 크기

(8, 4)
(8, 8)
(8,)


In [5]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0)
# 출력 시 값을 깔끔하게 해준다.

print(total_hidden_states) # (timesteps, output_dim)의 크기. 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.94396998 0.74713475 0.9954706  0.6690637  0.97140907 0.90954536
  0.94800985 0.66322223]
 [0.99977137 0.99827165 0.99998516 0.99959392 0.99989302 0.99994358
  0.9999436  0.99734752]
 [0.99997044 0.99981066 0.99999832 0.99995076 0.99999195 0.99999544
  0.999995   0.99931312]
 [0.99996387 0.99972646 0.99999676 0.999915   0.99999014 0.99999243
  0.9999923  0.99895209]
 [0.9999449  0.99970789 0.99999255 0.99991859 0.99998126 0.99998982
  0.99998614 0.99883912]
 [0.99989447 0.99953072 0.99998577 0.99990831 0.99997857 0.99998284
  0.99998382 0.99858357]
 [0.99995633 0.99968726 0.99999695 0.99995541 0.99996846 0.99999583
  0.99998976 0.99938193]
 [0.99995268 0.99960939 0.99999609 0.99992605 0.99998009 0.99999321
  0.9999901  0.99908459]
 [0.99991254 0.99957003 0.99998925 0.99993168 0.99996447 0.99998933
  0.99998227 0.99894937]
 [0.99988252 0.99956733 0.99999009 0.99994261 0.99998032 0.99998702
  0.99998858 0.99898045]

# 파이토치의 nn.RNN()
- 파이토치에서는 nn.RNN()을 통해서 RNN 셀을 구현

In [ ]:
import torch
import torch.nn as nn

In [ ]:
input_size = 5 # 입력의 크기
hidden_size = 8 # 은닉 상태의 크기

In [ ]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

In [ ]:
cell = nn.RNN(input_size, hidden_size, batch_first=True)

In [ ]:
outputs, _status = cell(inputs)

- RNN 셀은 두 개의 입력을 리턴하는데, 첫번째 리턴값은 모든 시점(timesteps)의 은닉 상태들이며, 두번째 리턴값은 마지막 시점(timestep)의 은닉 상태

In [ ]:
print(outputs.shape) # 모든 time-step의 hidden_state

torch.Size([1, 10, 8])


In [ ]:
torch.Size([1, 10, 8])

torch.Size([1, 10, 8])

In [ ]:
print(_status.shape) # 최종 time-step의 hidden_state

torch.Size([1, 1, 8])


In [ ]:
torch.Size([1, 1, 8])

torch.Size([1, 1, 8])

# 깊은 순환 신경망(Deep Recurrent Neural Network)

<img src='https://drive.google.com/uc?id=1nqzE9HMLvZDfKJNd1N_FfWH6bL2gr-zl' width=30%>

- 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 순환 신경망
- 깊은 순환 신경망을 파이토치로 구현할 때는 nn.RNN()의 인자인 num_layers에 값을 전달하여 층을 쌓습니다

In [ ]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)

In [ ]:
print(outputs.shape) # 모든 time-step의 hidden_state

torch.Size([1, 10, 8])


In [ ]:
print(_status.shape) # (층의 개수, 배치 크기, 은닉 상태의 크기)

torch.Size([1, 1, 8])


# 양방향 순환 신경망(Bidirectional Recurrent Neural Network)
- 양방향 순환 신경망은 시점 t에서의 출력값을 예측할 때 이전 시점의 데이터뿐만 아니라, 이후 데이터로도 예측할 수 있다는 아이디어에 기반
- Exercise is very effective at [          ] belly fat. 문제
- RNN이 과거 시점(time step)의 데이터들을 참고해서, 찾고자하는 정답을 예측하지만 실제 문제에서는 과거 시점의 데이터만 고려하는 것이 아니라 향후 시점의 데이터에 힌트가 있는 경우도 많습니다.
- 이전 시점의 데이터뿐만 아니라, 이후 시점의 데이터도 힌트로 활용하기 위해서 고안된 것이 양방향 RNN  
<img src='https://drive.google.com/uc?id=1_bKEF841DtDF7ez34PR_XWGFWjGc_5ne' width=50%>
- 하나의 출력값을 예측하기 위해 기본적으로 두 개의 메모리 셀을 사용
- 첫번째 메모리 셀은 앞에서 배운 것처럼 앞 시점의 은닉 상태(Forward States)를 전달받아 현재의 은닉 상태를 계산(주황색 셀)
- 두번째 메모리 셀은 앞 시점의 은닉 상태가 아니라 뒤 시점의 은닉 상태(Backward States)를 전달 받아 현재의 은닉 상태를 계산(초록색 셀)
- **은닉층을 무조건 추가한다고 해서 모델의 성능이 좋아지는 것은 아닙니다**
- **은닉층을 추가하면, 학습할 수 있는 양이 많아지지만 또한 반대로 훈련 데이터 또한 그만큼 많이 필요**

In [ ]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True, bidirectional = True)
outputs, _status = cell(inputs)

In [ ]:
print(outputs.shape) # (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2)

torch.Size([1, 10, 16])


In [ ]:
print(_status.shape) # (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)

torch.Size([4, 1, 8])
